In [12]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import accuracy_score
from sklearn import preprocessing

In [13]:
data = pd.read_csv("data/combined_expression.csv")

In [14]:
data.head()

,CELL_LINE_NAME,classification,TSPAN6,TNMD,DPM1,SCYL3,C1orf112,FGR,CFH,FUCA2,...,COL15A1,C6orf10,TMEM225,NOTCH4,PBX2,AGER,RNF5,AGPAT1,DFNB59,PRRT1
0,1240121,1,6.419526,3.182094,9.320548,3.759654,3.802619,3.215753,4.698729,7.873672,...,3.245454,2.953508,3.543429,3.352022,4.672310,3.641128,3.135310,3.737072,3.450927,3.168800
1,1240122,2,7.646494,2.626819,10.153853,3.564755,3.942749,3.290760,3.551675,8.252413,...,2.786709,3.077382,3.728232,3.208882,4.586840,3.395654,3.586800,3.519128,3.115323,3.051645
2,1240123,1,8.319417,3.111183,9.643558,4.757258,3.919757,3.602185,3.329644,9.076950,...,3.459089,3.085394,3.462811,3.339030,4.614897,3.395845,3.419193,3.971646,3.729310,3.320022
3,1240124,1,9.006994,3.028173,9.686700,4.280504,3.147646,3.188881,3.293807,8.678790,...,2.835403,2.960303,3.415083,3.290171,4.770123,3.400821,3.383734,3.798107,2.822404,3.297547
4,1240127,1,7.985676,2.694729,10.676134,4.159685,3.804637,3.481942,3.111261,7.555407,...,2.896523,2.849899,3.480114,3.226128,5.832710,3.612179,3.347095,4.457963,5.198524,4.553586


In [15]:
X = data.drop(columns=['CELL_LINE_NAME', 'classification'])
y = data['classification']
feat_labels = list(X.columns)

In [16]:
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(X)
X = pd.DataFrame(x_scaled)

In [17]:
# 20% test, 80% train
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [18]:
# create and train the classifier
clf = RandomForestClassifier(n_estimators=100, random_state=0, n_jobs=-1)
clf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=-1, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [19]:
# associating each feature with its relative importance
feat_importances = list(zip(feat_labels, clf.feature_importances_))

In [27]:
# sorting the feature importances
def sort_tuple(tup):
    tup.sort(key = lambda x: x[1], reverse=True)
    return tup
feat_importances = sort_tuple(feat_importances)

In [38]:
genes = [i[0] for i in feat_importances if i[1] >= 1e-3]
len(genes)

40

In [40]:
s_feats = pd.DataFrame(genes)
s_feats.to_csv('cleaned/selected_genes.csv', index=False)